# FairAdj
## Variational Graph AutoEncoder

In [1]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch_geometric.nn as Gnn
from torch_geometric.utils import train_test_split_edges

In [2]:
# NOTE: this dataset is just for test
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)

Processing...
Done!
/home/wsl/miniconda3/envs/GNN/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [3]:
# NOTE: TorchG has a VGAE template, where encoder needs to be explicitly given; if not giving decoder, inner product will be the default
class VariationalGCNEncoder(torch.nn.Module):
	def __init__(self, in_channels, out_channels):
		super(VariationalGCNEncoder,self).__init__()
		self.conv1 = Gnn.GCNConv(in_channels , 2 * out_channels, cached= True)
		self.conv_mu = Gnn.GCNConv(2 * out_channels, out_channels, cached= True)
		self.conv_logstd = Gnn.GCNConv(2 * out_channels, out_channels, cached= True)
	
	def forward(self, x, edge_index):
		x = self.conv1(x, edge_index)
		return self.conv_mu(x,edge_index), self.conv_logstd(x, edge_index)

out_c = 2
num_feat = dataset.num_features
model = Gnn.VGAE(VariationalGCNEncoder(num_feat,out_c))

In [4]:
model = model.cuda()
x = data.x.cuda()
train_pos_edge_index = data.train_pos_edge_index.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [5]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [14]:
epochs = 300
for epoch in range(1, epochs + 1):
    # VGAE training part
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    # Fair Imporving part
    z_mean = model.encode(x,train_pos_edge_index)
    print(z_mean.shape)
    # recovered = model.decode(z_mean,z_std)
    break


Epoch: 001, AUC: 0.7583, AP: 0.7680
torch.Size([3327, 2])


3703